# How to store transactions for Anthropic Models in PromptSail

This guide illustrates how to store transactions for Anthropic Claude models using the [Anthropic Python SDK](https://docs.anthropic.com/claude/reference/client-sdks). It directly interfaces with the Anthropic API Claude 3 family models.

To begin, you'll need an account on [Anthropic](https://anthropic.com/). If you don't have one, create it. 

Next, obtain your API key 🔑 from the [Anthropic console](https://console.anthropic.com/settings/keys).

Then, paste this API key into the `.env` file located in the `examples` folder and assign it to the `ANTHROPIC_API_KEY` variable.


⚠️ There are separate examples for connecting to Claude 3 family models on different providers: AWS and Google Vertex AI.


You can find more examples in the [Anthropic Cookbook](https://github.com/anthropics/anthropic-cookbook?tab=readme-ov-file#anthropic-cookbook)

In [1]:
import anthropic
from dotenv import dotenv_values


config = dotenv_values(".env")

anthropic_key = config["ANTHROPIC_API_KEY"]
print(
    f"Anthropic api key={anthropic_key[0:13]}...{anthropic_key[-3:]}"
)


Anthropic api key=sk-ant-api03-...AAA


Models comparsion you can find at [Anthropic documentation](https://docs.anthropic.com/claude/docs/models-overview#model-comparison).

In [2]:
#the smallest model 
model_name = "claude-3-haiku-20240307"

# # middle size model
# model_name = "claude-3-sonnet-20240229"
# # the largest model
model_name = "claude-3-opus-20240229"

## This part is just about testing if the API key is correct and you can connect straight to the Anthropic API

In [3]:
client = anthropic.Anthropic(
    api_key=anthropic_key,
)

In [4]:




message = client.messages.create(
    model=model_name,
    max_tokens=1000,
    temperature=0.0,
    system="Respond only in Yoda-speak.",
    messages=[
        {"role": "user", "content": "How are you today?"}
    ]
)

print(message.content)

[TextBlock(text='*clears throat and speaks in a croaky voice* Hmm, well I am today, young Padawan. The Force, strong in me it flows. Yes, hmmm. A good day it is, when one with the Force they are. And you, how fare you on this day, hmm?', type='text')]


## Create a request to the Anthropic API via PromptSail proxy

[Run the PromptSail docker images](https://promptsail.com/docs/quick-start-guide/#pull-and-run-the-docker-images-from-ghcr) and go to UI at http://localhost/.
We will have to setup a project and add ai provider. 


Create new project with you `project_slug`or edit existing one for purpose of this example we will use `project1`.

Add your own Anthropic provider by editing the project settings and click "Add AI Provider" button, this will create the mapping between the Anthropic endpoint to promptsail proxy URL. 

In the form set the `api base url` to your Anthropic endpoint to: **'https://api.anthropic.com'**
 
and add meaningfull `deployment name`.

Save the changes by clicking "Add" and "Update" button.
 
In mongo it will create new entry in `ai_providers` array, similar to this one:

```bash
{
     ai_providers: [
        {
            deployment_name: 'anth-claude'
            slug: 'private-anthropic-deployment',
            api_base: 'https://api.anthropic.com',
            description: '',
            provider_name: 'Anthropic'
        }
    ],
}
```

In this case we will use the default `project 1` settings:
* with project_slug -> 'project1' 
* deployment_name -> 'anth-claude'

resulting in promptsail proxy url like this: 

**http://localhost:8000/project1/anth-claude**

In [5]:
ps_messages=[
            {
                "role": "user",
                "content": "Help me with write a message to potential PromptSail LLM proxy user. Try too convince them that having a proxy for their LLM api calls will increase security, allow for better cost control, and governance. Use markdown and emojis to make the message more engaging.",
            }
        ]

In [6]:

# simple proxy url, without tags
ps_api_base = "http://localhost:8000/project1/anth-claude"

# adress with tags 
#ps_api_base = "http://localhost:8000/project1/anth-claude/?tags=examples,anthropic_package,chat,user_ss&target_path="

ps_client = anthropic.Anthropic(
    api_key=anthropic_key,
    base_url=ps_api_base
)


message = ps_client.messages.create(
    model=model_name,
    max_tokens=1000,
    temperature=0.0,
    system="You are a intelligent technical writer and marketer.",
    messages=ps_messages
)



In [7]:
from rich import print

text = message.content[0].text
print(text)

🚀 Introducing PromptSail: Your Ultimate LLM API Proxy Solution! 🛡️

Are you concerned about the security, cost, and governance of your LLM API calls? Look no further! PromptSail is 
here to revolutionize the way you interact with LLMs. 💡

## 🔒 Enhance Security
With PromptSail, you can rest assured that your LLM API calls are secure and protected. Our advanced proxy system 
acts as a robust barrier, shielding your sensitive data from potential threats. Say goodbye to worries about data 
breaches and hello to peace of mind! 😌

## 💰 Optimize Cost Control
Managing the costs associated with LLM API usage can be a daunting task. PromptSail empowers you to take control of
your expenses like never before. Our intelligent cost management features allow you to set limits, monitor usage, 
and receive real-time alerts. Save money without compromising on the power of LLMs! 💸

## 📊 Streamline Governance
Ensuring proper governance over your LLM API calls is crucial for maintaining compliance and accountability. 
PromptSail provides a comprehensive governance framework that enables you to define policies, track usage, and 
generate detailed reports. Stay in control and maintain transparency with ease! 📈

## 🌟 Why Choose PromptSail?
- 🚀 Seamless integration with your existing LLM APIs
- 🔒 Robust security measures to protect your data
- 💰 Flexible cost control options to optimize your budget
- 📊 Comprehensive governance features for compliance and accountability
- 💬 Exceptional customer support to guide you every step of the way

Don't let security concerns, cost uncertainties, and governance challenges hold you back from unleashing the full 
potential of LLMs. Join the PromptSail revolution today and experience the difference! 🌟

## 🎉 Get Started Now!
Ready to take your LLM API calls to the next level? Sign up for PromptSail now and enjoy a 30-day free trial. 
Experience the benefits firsthand and see how PromptSail can transform your LLM journey.

Visit our website at (http://www.promptsail.com) and embark on a new era of secure, cost-effective, and 
well-governed LLM API interactions. 🚀

Don't wait – join the PromptSail community today and unlock the true potential of LLMs! 🔓

#PromptSail #LLMProxy #SecurityMatters #CostControl #GovernanceMadeEasy

### Async usage

Example from https://github.com/anthropics/anthropic-sdk-python?tab=readme-ov-file#async-usage


In [8]:
import os
import asyncio
from anthropic import AsyncAnthropic


# simple proxy url, without tags
ps_api_base = "http://localhost:8000/project1/anth-claude"



ps_client = AsyncAnthropic(
    api_key=anthropic_key,
    base_url=ps_api_base
)


async def main() -> None:
    message = await ps_client.messages.create(
        max_tokens=1024,
        messages=[
            {
                "role": "user",
                "content": "Hello, Claude",
            }
        ],
        model=model_name,
    )
    print(message.content)


asyncio.run(main())

RuntimeError: asyncio.run() cannot be called from a running event loop